In [ ]:
# Set a range of dayobs values to search - 
day_obs_min = "Today"
#day_obs_min = "20250609"
day_obs_max = "Today"
#day_obs_max = "20250609"
time_order = 'newest first'
show_salIndex = 'all'
show_table = True
show_timeline = False

# EFD Scripts + Logs 

In [ ]:
try:
    from rubin_nights import connections
except ModuleNotFoundError:
    print("Installing rubin_nights")
    !pip install --user --upgrade git+https://github.com/lsst-sims/rubin_nights.git --no-deps > /dev/null 2>&1
# Could also install if needed to upgrade ..

In [ ]:
import os
from astropy.time import Time, TimeDelta
import astropy.units as u
import pandas as pd
from IPython.display import display, HTML

from rubin_nights import connections
from rubin_nights import scriptqueue
from rubin_nights import scriptqueue_formatting
import rubin_nights.dayobs_utils as rn_dayobs


import logging
logging.getLogger('rubin_nights').setLevel(logging.INFO)

if os.getenv("EXTERNAL_INSTANCE_URL") is None:
    tokenfile = '/Users/lynnej/.lsst/usdf_rsp'
    site = 'usdf'
else:
    tokenfile = None
    site = None

In [ ]:
if isinstance(day_obs_min, str):
    if day_obs_min.lower() == "today":
        day_obs_min = rn_dayobs.day_obs_str_to_int(rn_dayobs.today_day_obs())
    elif day_obs_min.lower() == "yesterday":
        day_obs_min = rn_dayobs.day_obs_str_to_int(rn_dayobs.yesterday_day_obs())
if isinstance(day_obs_max, str):
    if day_obs_max.lower() == "today":
        day_obs_max = rn_dayobs.day_obs_str_to_int(rn_dayobs.today_day_obs())
    elif day_obs_max.lower() == "yesterday":
        day_obs_max = rn_dayobs.day_obs_str_to_int(rn_dayobs.yesterday_day_obs())

try:
    t_start = Time(f"{rn_dayobs.day_obs_int_to_str(day_obs_min)}T12:00:00", format='isot', scale='utc')
except ValueError:
    print(f"Is day_obs_min the right format? {day_obs_min} should be YYYYMMDD")
    t_start = None
try:
    t_end = Time(f"{rn_dayobs.day_obs_int_to_str(day_obs_max)}T12:00:00", format='isot', scale='utc') + TimeDelta(1, format='jd')
except ValueError:
    print(f"Is day_obs_max the right format? {day_obs_max} should be YYYYMMDD")
    t_start = None

if t_start is None or t_end is None:
    print("Did not get valid inputs for time period.")


print(f"Querying for messages from {t_start.iso} to {t_end.iso}")
print(f"Notebook executed at {Time.now().utc.iso}")
endpoints = connections.get_clients(tokenfile=tokenfile, site=site)
efd_and_messages, cols = scriptqueue.get_consolidated_messages(t_start, t_end, endpoints)

# Could add these to parameters
save_log = False
make_link = False

if save_log:
    log_filename = f"log_{day_obs_min}_{day_obs_max}.h5"
    # We will always get a performance warning here, because the dataframe includes string objects
    with warnings.catch_warnings():
        warnings.simplefilter("ignore")
        efd_and_messages[cols].to_hdf(log_filename, key='messages')
        print(f"Wrote to {log_filename}")
if make_link:
    import base64
    html_table = efd_and_messages[cols].to_xml(index=False)
    b64 = base64.b64encode(html_table.encode())
    payload = b64.decode()
    log_xml =  f"log_{day_obs_min}_{day_obs_max}.xml"
    html_link = f'<a download="{log_xml}" href="data:text/csv;base64,{payload}" target="_blank">Download XML table of log messages</a>'
    display(HTML(html_link))
    print(" read download with pandas.read_xml, convert times using .astype('datetime64[ns]')")

In [ ]:
if isinstance(show_salIndex, str) and show_salIndex.lower() == 'all':
    show_salIndex = efd_and_messages.salIndex.unique()
# Ok, otherwise we have to do some parsing .. we get a string but need list of ints.
showsal = []
for i in show_salIndex:
    try:
        showsal.append(int(i))
    except ValueError:
        # Wasn't an integer, pass
        # easier when no negative int salIndexes.
        pass
show_salIndex = showsal

if show_table:
    html = scriptqueue_formatting.format_html(efd_and_messages, cols=cols, time_order=time_order, show_salIndex=show_salIndex)
    display(HTML(html))

# Timeline

In [ ]:
# extra_paths = (
#     "/sdf/data/rubin/shared/scheduler/packages/rubin_scheduler-3.4.1.dev2+g9f70dc6.d20241204",
#     "/sdf/data/rubin/shared/scheduler/packages/schedview-0.15.1.dev97+g5d2f010",
# )
# for extra_path in extra_paths:
#     sys.path.insert(0, extra_path)

# import schedview.plot.timeline

# import bokeh.io
# import bokeh.models

# bokeh.io.output_notebook(hide_banner=True)

In [ ]:
# # Customize schedview's TimelinePlotter for the efd_and_messages DataFrame:

# class EvaluateScriptQueueTimeline(schedview.plot.timeline.TimelinePlotter):
#     key = "evaluate_script_queue"
#     hovertext_column = "html"
#     default_figure_kwargs: dict = {
#         "x_axis_type": "datetime",
#         "y_range": bokeh.models.FactorRange(),
#         "height": 64,
#         "width": 1536,
#     }

#     @classmethod
#     def _create_source(cls, data, *args, **kwargs) -> bokeh.models.ColumnDataSource:

#         # Bokeh can't deal with NaT in a time column, so replace it with a magic value.
#         munged = data.copy()
#         timestamp_cols = [c for c in data if c.startswith("timestamp")]
#         for col in timestamp_cols:
#             munged[col] = munged[col].fillna(pd.Timestamp("1970-01-01", tz="UTC"))

#         munged["salIndexName"] = [
#             get_name_and_color_from_salindex(i)[0] for i in data["salIndex"]
#         ]

#         source = super()._create_source(munged, *args, **kwargs)
#         return source

#     @classmethod
#     def _make_hovertext(cls, row_data: pd.Series) -> str:
#         explicitly_shown = ("name", "time", "description", "config")
#         table_columns = [
#             f
#             for f in row_data.index
#             if (f not in explicitly_shown)
#             and (str(row_data[f]) != "")
#             and (row_data[f] is not None)
#             and (not (isinstance(row_data[f], float) and np.isnan(row_data[f])))
#             and (
#                 not (isinstance(row_data[f], pd.Timestamp) and row_data[f].year <= 1970)
#             )
#         ]

#         d = row_data["timestampProcessEnd"]

#         table_data = row_data.copy()
#         hovertext = f"""<h1>{row_data['name']} at {row_data['time']}</h1>
#         {row_data[table_columns].to_frame().to_html(header=False)}
#         <h2>Description</h2>
#         <div id="" style="overflow:auto; height:auto; width:512">
#         <pre>{format_tracebacks(row_data)}</pre>
#         </div>
#         <h2>Config</h2>
#         <p>{format_config_as_yaml_with_colors(row_data)}</p>
#         """
#         return hovertext

#     @property
#     def _make_glyph_kwargs(self) -> dict:

#         sal_indexes = tuple(range(np.max(self.source.data["salIndex"]) + 1))
#         color_map = bokeh.transform.factor_cmap(
#             "salIndexName",
#             palette=[get_name_and_color_from_salindex(i)[1] for i in sal_indexes],
#             factors=[get_name_and_color_from_salindex(i)[0] for i in sal_indexes],
#         )

#         glyph_kwargs = {
#             "x": self.time_column,
#             "y": self.factor_column,
#             "size": 10,
#             "line_color": color_map,
#             "fill_color": color_map,
#             "marker": "diamond",
#         }

#         return glyph_kwargs

In [ ]:
# # A minor modification will split the different `salIndex`es along different parallel timelines:

# class SplitEvaluateScriptQueueTimeline(EvaluateScriptQueueTimeline):
#     factor_column = "salIndexName"
#     default_figure_kwargs: dict = {
#         "x_axis_type": "datetime",
#         "y_range": bokeh.models.FactorRange(),
#         "height": 128,
#         "width": 1536,
#     }

In [ ]:
# if show_timeline:
#     display(HTML('<h2 id="timeline">Timeline</h2>'))
#     display(HTML("""<p>Be sure to use the magnification tool (under the three dots on the right,
#     the box with magnifying glass) to expand the data points of interest enough to separate them.</p>
#     <p>Hover over datapoints to get additional details."""))

#     msg = ["Color coding by "]
#     for i in np.sort(efd_and_messages.salIndex.unique()):
#         what, color = get_name_and_color_from_salindex(i)
#         msg.append(f" <font style='background-color: {color[0:]};'>{what}</font> ")
#     display(HTML(" ".join(msg)))

#     split_script_timeline = SplitEvaluateScriptQueueTimeline(efd_and_messages)
#     bokeh.io.show(split_script_timeline.plot)